In [1]:
# nbi:hide_in
##########################
# Created on Oct 2020
# @author: juans
##########################

In [2]:
# nbi:hide_in
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

In [3]:
# nbi:hide_in
def mag2db(x):
    y = 20 * np.log10( np.abs( x + np.finfo(float).eps ) )
    return y
    
def db2mag(x):
    y = 10.0**(x / 20)
    return y 

In [4]:
# nbi:hide_in
class LevelEstimator:
    def __init__(self, tauAttack, tauRelease, fs):
        self.b0Att = 1 - np.exp(-1 / (tauAttack * fs))
        self.b0Rel = 1 - np.exp(-1 / (tauRelease * fs))
        self.state = 0
    
    def process(self, x):
        if np.abs(x) > self.state:
            self.state += self.b0Att * (np.abs(x) - self.state)
        else:
            self.state += self.b0Rel * (np.abs(x) - self.state)
        
        y = self.state
        
        return y
    
    def getState(self):
        return self.state
    
    def setState(self, newState):
        self.state = newState

In [5]:
# nbi:hide_in
class Compressor:
    def __init__(self, tauAttack, tauRelease, threshold, ratio, fs):
        self.estimator = LevelEstimator(tauAttack, tauRelease, fs)
        self.th = threshold
        self.ratio = ratio
        self.level = self.estimator.getState()
        self.gain = self.computeGain(self.level)
        
    def computeGain(self, level):
        leveldB = mag2db(level)
        gainIndB = np.max((leveldB - self.th, 0.0)) * (1.0 / self.ratio - 1.0)
        gain = db2mag(gainIndB)
        return gain
    
    def process(self, x):
        self.level = self.estimator.process(x)
        self.gain = self.computeGain(self.level)
        y = x * self.gain
        return y
    
    def getLevel(self):
        return self.estimator.getState()
    
    def setLevel(self, newState):
        self.estimator.setState(newState)
        self.level = self.estimator.getState()
        self.gain = self.computeGain(self.level)
        
    def getGain(self):
        return self.gain
        
    def clear(self):
        self.setLevel(0)
        

In [6]:
# nbi:hide_in
gainBeforeSlider = widgets.IntSlider(
    value=-20,
    min=-20,
    max=0,
    step=1,
    description='level Before',
    readout_format='d',
    continuous_update=False
)

gainDuringSlider = widgets.IntSlider(
    value=0,
    min=-20,
    max=6,
    step=1,
    description='level During',
    readout_format='d',
    continuous_update=False
)

gainAfterSlider = widgets.IntSlider(
    value=-20,
    min=-20,
    max=0,
    step=1,
    description='level Before',
    readout_format='d',
    continuous_update=False
)

In [9]:
# nbi:hide_in
fs = 10000
tauAtt = 100./1000
tauRel = 100./1000
threshold = -10.0
ratio = 2.

dur = 2
t = np.linspace(0, dur, dur * fs, endpoint=False)
comp = Compressor(tauAtt, tauRel, threshold, ratio, fs)   

def plot(gainBefore, gainDuring, gainAfter):
    
    x = np.random.uniform(low=-1.0, high=1.0, size=np.size(t))
    e = db2mag(gainDuring) * np.ones_like(x)
    e[t < 0.5] = db2mag(gainBefore)
    e[t > 1] = db2mag(gainAfter)
    x *= e
    
    y = np.zeros_like(x)
    l = np.zeros_like(x)
    g = np.zeros_like(x)
    
    comp.clear()
    for i in np.arange(np.size(x)):
        y[i] = comp.process(x[i])
        l[i] = comp.getLevel()
        g[i] = comp.getGain()
        
    plt.figure(1, figsize=(12, 12))
    plt.subplot(411)
    plt.plot(t, x)
    plt.xlim(t[[0,-1]])
    plt.ylim([-1, 1])
    plt.grid(True)
    plt.title("Input Signal")
    
    plt.subplot(412)
    plt.plot(t, l)
    plt.hlines(db2mag(threshold), t[0], t[-1])
    plt.xlim(t[[0,-1]])
    plt.ylim([0, 1])
    plt.grid(True)
    plt.title("Estimated Level")
    
    plt.subplot(413)
    plt.plot(t, g)
    plt.xlim(t[[0,-1]])
    plt.ylim([0.5, 1.2])
    plt.grid(True)
    plt.title("Gain Function")
    
    plt.subplot(414)
    plt.plot(t, y)
    plt.xlim(t[[0,-1]])
    plt.ylim([-1, 1])
    plt.grid(True)
    plt.title("Output Signal")
    plt.tight_layout()
    
widgets.interactive(plot, gainBefore=gainBeforeSlider, gainDuring=gainDuringSlider, gainAfter=gainAfterSlider)

interactive(children=(IntSlider(value=-8, continuous_update=False, description='level Before', max=0, min=-20)…